<a href="https://colab.research.google.com/github/oluwoleowoeye/food-price-analysis/blob/main/Food_price_anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd

# 1. CORRECT GITHUB URL (use RAW link)
github_raw_url = "https://github.com/oluwoleowoeye/food-price-analysis/raw/main/Project_food%20prices%20anomalies%20indicator.xlsx"

# 2. LOAD EXCEL WITH EXPLICIT ENGINE
try:
    df = pd.read_excel(github_raw_url, sheet_name='Goal2', engine='openpyxl')  # 'Goal2' is case-sensitive!
    print("Data loaded successfully!")
    print(df.head())
except Exception as e:
    print("Error:", e)

Data loaded successfully!
   Goal Target Indicator   SeriesCode  \
0     2    2.c     2.c.1  AG_FPA_CFPI   
1     2    2.c     2.c.1  AG_FPA_CFPI   
2     2    2.c     2.c.1  AG_FPA_CFPI   
3     2    2.c     2.c.1  AG_FPA_CFPI   
4     2    2.c     2.c.1  AG_FPA_CFPI   

                                   SeriesDescription  GeoAreaCode  \
0  Indicator of Food Price Anomalies (IFPA), by C...            4   
1  Indicator of Food Price Anomalies (IFPA), by C...            4   
2  Indicator of Food Price Anomalies (IFPA), by C...            4   
3  Indicator of Food Price Anomalies (IFPA), by C...            4   
4  Indicator of Food Price Anomalies (IFPA), by C...            4   

   GeoAreaName  TimePeriod  Value  Time_Detail  ...  BasePeriod  \
0  Afghanistan        2010  -0.12         2010  ...         NaN   
1  Afghanistan        2011  -0.17         2011  ...         NaN   
2  Afghanistan        2012  -0.28         2012  ...         NaN   
3  Afghanistan        2013   0.30         20

In [36]:

print("Missing values in Main Data (Goal2):")
print(df_main.isnull().sum())
# --- Handle missing values ---
df_main_cleaned = df_main.copy()
# A. Fill numerical columns (e.g., 'Value', 'UpperBound', 'LowerBound')
numerical_cols = ['Value', 'UpperBound', 'LowerBound']
for col in numerical_cols:
    df_main_cleaned[col] = df_main_cleaned[col].fillna(df_main_cleaned[col].mean())

# B. Fill categorical columns (e.g., 'Observation Status', 'Reporting Type')
categorical_cols = ['Observation Status', 'Reporting Type', 'Units']
for col in categorical_cols:
    df_main_cleaned[col] = df_main_cleaned[col].fillna("Unknown")

# C. Drop rows if critical columns are missing (e.g., 'SeriesCode', 'TimePeriod')
critical_cols = ['SeriesCode', 'TimePeriod', 'GeoAreaCode']
df_main_cleaned = df_main_cleaned.dropna(subset=critical_cols)  # Fixed typo: 'dropha' → 'dropna'
# Verify cleaning
print("\nMissing values after cleaning:")
print(df_main_cleaned.isnull().sum())  # Fixed typo: 'df main cleaned' → 'df_main_cleaned'
# Additional verification
print("\nSample of cleaned data:")
print(df_main_cleaned.head(3))

Missing values in Main Data (Goal2):
Goal                           0
Target                         0
Indicator                      0
SeriesCode                     0
SeriesDescription              0
GeoAreaCode                    0
GeoAreaName                    0
TimePeriod                     0
Value                        285
Time_Detail                    0
TimeCoverage                5817
UpperBound                  5817
LowerBound                  5817
BasePeriod                  5817
Source                       285
GeoInfoUrl                  5817
FootNote                       0
Nature                       285
Observation Status             0
Reporting Type                 0
Severity of price levels    5154
Type of product             3477
Units                          0
dtype: int64

Missing values after cleaning:
Goal                           0
Target                         0
Indicator                      0
SeriesCode                     0
SeriesDescription          

In [42]:
# Merge using the correct key columns:
merged_df = pd.merge(
    df_main_cleaned,
    df_codes,
    left_on='SeriesCode',   # Column in df_main_cleaned
    right_on='Code',        # Column in df_codes (previously called 'CodeColumn')
    how='left'
)

# Verify the merge
print("Merged Data Columns:", merged_df.columns.tolist())
print("\nSample Rows:")
print(merged_df.head())

Merged Data Columns: ['Goal', 'Target', 'Indicator', 'SeriesCode', 'SeriesDescription', 'GeoAreaCode', 'GeoAreaName', 'TimePeriod', 'Value', 'Time_Detail', 'TimeCoverage', 'UpperBound', 'LowerBound', 'BasePeriod', 'Source', 'GeoInfoUrl', 'FootNote', 'Nature', 'Observation Status', 'Reporting Type', 'Severity of price levels', 'Type of product', 'Units', 'Dimension', 'Code', 'Description']

Sample Rows:
   Goal Target Indicator   SeriesCode  \
0     2    2.c     2.c.1  AG_FPA_CFPI   
1     2    2.c     2.c.1  AG_FPA_CFPI   
2     2    2.c     2.c.1  AG_FPA_CFPI   
3     2    2.c     2.c.1  AG_FPA_CFPI   
4     2    2.c     2.c.1  AG_FPA_CFPI   

                                   SeriesDescription  GeoAreaCode  \
0  Indicator of Food Price Anomalies (IFPA), by C...            4   
1  Indicator of Food Price Anomalies (IFPA), by C...            4   
2  Indicator of Food Price Anomalies (IFPA), by C...            4   
3  Indicator of Food Price Anomalies (IFPA), by C...            4   
4 

In [43]:
from sklearn.preprocessing import MinMaxScaler

# Normalize 'Value' to [0, 1]
scaler = MinMaxScaler()
merged_df['Value_Normalized'] = scaler.fit_transform(merged_df[['Value']])

# Optional: Standardize 'UpperBound' (mean=0, std=1)
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# merged_df['UpperBound_Standardized'] = scaler.fit_transform(merged_df[['UpperBound']])

print("\nNormalized Data Sample:")
print(merged_df[['Value', 'Value_Normalized']].head())


Normalized Data Sample:
   Value  Value_Normalized
0  -0.12          0.357010
1  -0.17          0.356689
2  -0.28          0.355982
3   0.30          0.359707
4   0.42          0.360478


In [46]:
merged_df.to_excel("Cleaned_Data.xlsx", index=False)

In [47]:
# Identify columns with a high percentage of missing values
missing_values_count = merged_df.isnull().sum()
total_rows = len(merged_df)
missing_values_percentage = (missing_values_count / total_rows) * 100

# Set a threshold for dropping columns (e.g., drop if more than 50% missing)
# Also explicitly include columns identified as having almost all missing values
columns_to_drop = missing_values_percentage[missing_values_percentage > 50].index.tolist()
explicitly_empty_cols = ['TimeCoverage', 'UpperBound', 'BasePeriod', 'GeoInfoUrl']
columns_to_drop.extend(explicitly_empty_cols)

# Remove duplicates in case any were added twice
columns_to_drop = list(set(columns_to_drop))

print("Columns to be dropped due to high missing values:")
print(columns_to_drop)

# Drop the identified columns from the merged_df
merged_df_cleaned_cols = merged_df.drop(columns=columns_to_drop)

# Verify the columns remaining
print("\nColumns remaining after dropping:")
print(merged_df_cleaned_cols.columns.tolist())

# Verify missing values in the new dataframe
print("\nMissing values in dataframe after dropping columns:")
print(merged_df_cleaned_cols.isnull().sum())

Columns to be dropped due to high missing values:
['LowerBound', 'Severity of price levels', 'Code', 'GeoInfoUrl', 'UpperBound', 'BasePeriod', 'TimeCoverage', 'Dimension', 'Type of product', 'Description']

Columns remaining after dropping:
['Goal', 'Target', 'Indicator', 'SeriesCode', 'SeriesDescription', 'GeoAreaCode', 'GeoAreaName', 'TimePeriod', 'Value', 'Time_Detail', 'Source', 'FootNote', 'Nature', 'Observation Status', 'Reporting Type', 'Units', 'Value_Normalized']

Missing values in dataframe after dropping columns:
Goal                    0
Target                  0
Indicator               0
SeriesCode              0
SeriesDescription       0
GeoAreaCode             0
GeoAreaName             0
TimePeriod              0
Value                   0
Time_Detail             0
Source                285
FootNote                0
Nature                285
Observation Status      0
Reporting Type          0
Units                   0
Value_Normalized        0
dtype: int64


In [49]:
# Save the cleaned data (with columns dropped) to an Excel file (.xlsx)
output_excel_path_cleaned = 'cleaned_food_price_data.xlsx'
# Ensure openpyxl is installed if not already: !pip install openpyxl
merged_df_cleaned_cols.to_excel(output_excel_path_cleaned, index=False, engine='openpyxl')

print(f"Cleaned data (with columns dropped) saved to '{output_excel_path_cleaned}'")

Cleaned data (with columns dropped) saved to 'cleaned_food_price_data.xlsx'
